# Importing libraries and import MNIST dataset

In [1]:
# importing libraries

import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm # for progress bar

# Libraries for TensorFlow
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers
from tensorflow import keras

# Library for Transfer Learning
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

print("Importing libraries completed.")

Importing libraries completed.


In [2]:
# Loading dataset from keras

(xtrain,ytrain),(xtest,ytest)= keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
# Loading dataset from keras

(xtrain,ytrain),(xtest,ytest)= keras.datasets.mnist.load_data()
print(type(xtrain))

<class 'numpy.ndarray'>


# Preprocessing images

In [4]:
xtrain=np.dstack([xtrain] * 3)
xtest=np.dstack([xtest]*3)
xtrain.shape,xtest.shape

((60000, 28, 84), (10000, 28, 84))

In [5]:
xtrain = xtrain.reshape(-1, 28,28,3)
xtest= xtest.reshape (-1,28,28,3)
xtrain.shape,xtest.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [6]:
from tensorflow.keras.utils import img_to_array, array_to_img

In [7]:
# Resize the images 48*48 as required by VGG16


xtrain = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtrain])
xtest = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtest])

xtrain.shape, xtest.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [8]:
# to store array value of the images
x=xtrain
# to store the labels of the images
y=ytrain

test_images=xtest
test_images_Original=xtest
# to store the labels of the images
test_image_label=ytest

val_images=xtest
val_images_Original=xtest
# to store the labels of the images
val_image_label=ytest # to store the labels of the images

In [9]:

# Training Dataset
print("Training Dataset")

x=np.array(x) # Converting to np arrary to pass to the model
print(x.shape)
y=to_categorical(y) # onehot encoding of the labels
#print(y)
print(y.shape)

# Test Dataset
print("Test Dataset")

test_images=np.array(test_images) 
print(test_images.shape)

test_image_label=to_categorical(test_image_label) # onehot encoding of the labels)
print(test_image_label.shape)

# Validation Dataset
print("Validation Dataset")

val_images=np.array(val_images) 
print(val_images.shape)

val_image_label=to_categorical(val_image_label) # onehot encoding of the labels)
print(val_image_label.shape)



Training Dataset
(60000, 48, 48, 3)
(60000, 10)
Test Dataset
(10000, 48, 48, 3)
(10000, 10)
Validation Dataset
(10000, 48, 48, 3)
(10000, 10)


In [10]:

x = x[:6000,:]
y = y[:6000,:]
print(x.shape, y.shape)
val_images, val_image_label = val_images[:1000,:], val_image_label[:1000,:]
print(val_images.shape, val_image_label.shape)
test_images, test_image_label = test_images[1000:2000,:], test_image_label[1000:2000,:]
print(test_images.shape, test_image_label.shape)


(6000, 48, 48, 3) (6000, 10)
(1000, 48, 48, 3) (1000, 10)
(1000, 48, 48, 3) (1000, 10)


# Load pretrained model

In [11]:
# Check properties of the model that we are going to use for Transfer Learning

print("Summary of default VGG16 model.\n")

# we are using VGG16 for transfer learnin here. So we have imported it
from tensorflow.keras.applications import VGG16

# initializing model with weights='imagenet'i.e. we are carring its original weights
model_vgg16=VGG16(weights='imagenet')

# display the summary to see the properties of the model
model_vgg16.summary()

Summary of default VGG16 model.

553467096/553467096 [==============================] - 15s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                          

# Transfer Learning

In [12]:
# Modelling WITH Transfer Learning

# Here we will prepare model as per our requirements

print("Summary of Custom VGG16 model.\n")
print("1) We setup input layer and 2) We removed top (last) layer. \n")

# let us prepare our input_layer to pass our image size. default is (224,224,3). we will change it to (224,224,3)
input_layer=layers.Input(shape=(48,48,3))

# initialize the transfer model VGG16 with appropriate properties per our need.
# we are passing paramers as following
# 1) weights='imagenet' - Using this we are carring weights as of original weights.
# 2) input_tensor to pass the VGG16 using input_tensor
# 3) we want to change the last layer so we are not including top layer
model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)
model_vgg16.summary()

Summary of Custom VGG16 model.

1) We setup input layer and 2) We removed top (last) layer. 

58889256/58889256 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None

In [ ]:
# access the current last layer of the model and add flatten and dense after it

print("Summary of Custom VGG16 model.\n")

last_layer=model_vgg16.output # we are taking last layer of the model

# Add flatten layer: we are extending Neural Network by adding flatten layer
flatten=layers.Flatten()(last_layer)
dropout=layers.Dropout(0.3)(flatten)
# Add dense layer
dense1=layers.Dense(100,activation='relu')(dropout)

# Add dense layer to the final output layer
output_layer=layers.Dense(10,activation='softmax')(dense1)

# Creating model with input and output layer
model=models.Model(inputs=input_layer,outputs=output_layer)

# Summarize the model
model.summary()

In [15]:
# we will freez all the layers except the last layer

# we are making all the layers intrainable except the last 2 layers
print("We are making all the layers intrainable except the last layer. \n")

for layer in model.layers[:-1]:
    layer.trainable=False

model.layers[len(model.layers)-2].trainable = True


We are making all the layers intrainable except the last layer. 



# Compile model

In [16]:
# Compiling Model

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Model compilation completed.


# Evaluation before training

In [17]:
score = model.evaluate(test_images, test_image_label)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

32/32 [==============================] - 9s 21ms/step - loss: 14.0171 - accuracy: 0.1060
Test loss:  14.01707649230957
Test accuracy:  0.10599999874830246


# Fit the model

In [18]:
# Fit the Model

# xtrain2=xtrain.reshape(60000,48,48,3)
# xtest2=xtest.reshape(10000,48,48,3)

history = model.fit(x,y,epochs=20,batch_size=512,verbose=True,validation_data=(val_images,val_image_label))

print("Fitting the model completed.")

Epoch 1/20
12/12 [==============================] - 10s 599ms/step - loss: 11.4862 - accuracy: 0.2335 - val_loss: 3.6622 - val_accuracy: 0.5200
Epoch 2/20
12/12 [==============================] - 2s 153ms/step - loss: 4.7053 - accuracy: 0.4907 - val_loss: 2.2848 - val_accuracy: 0.6480
Epoch 3/20
12/12 [==============================] - 2s 154ms/step - loss: 2.9656 - accuracy: 0.6123 - val_loss: 1.5656 - val_accuracy: 0.7060
Epoch 4/20
12/12 [==============================] - 2s 154ms/step - loss: 2.2631 - accuracy: 0.6658 - val_loss: 1.2800 - val_accuracy: 0.7470
Epoch 5/20
12/12 [==============================] - 2s 155ms/step - loss: 1.8042 - accuracy: 0.6967 - val_loss: 1.1034 - val_accuracy: 0.7680
Epoch 6/20
12/12 [==============================] - 2s 155ms/step - loss: 1.5832 - accuracy: 0.7195 - val_loss: 0.9964 - val_accuracy: 0.7860
Epoch 7/20
12/12 [==============================] - 2s 155ms/step - loss: 1.4358 - accuracy: 0.7353 - val_loss: 0.8829 - val_accuracy: 0.7880
Epoc

# Evaluation after training

In [19]:
score = model.evaluate(test_images, test_image_label)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

32/32 [==============================] - 0s 13ms/step - loss: 0.5622 - accuracy: 0.8400
Test loss:  0.5621785521507263
Test accuracy:  0.8399999737739563
